In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 34.6 MB/s 
     |████████████████████████████████| 61 kB 492 kB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [3]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
import numpy as np

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
'''task_a_hyp = "This is offensive speech."
task_b_hyp = "This is targeted offense."
task_c_hyp_1= "This is targeted towards an individual."
task_c_hyp_2 = "This is targeted towards a group."
task_c_hyp_3 = "This isn't targeted towards a group or an individual."
f = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/olid_train_v2.csv')
f_a = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/bert_nli.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split('\t')
  tweet_id = row[0]
  tweet_text = ' '.join(row[1:-3])
  is_offensive = row[-3]
  is_targeted = row[-2]
  target = row[-1]
  if is_offensive == 'OFF':
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == 'TIN':
      f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
      if target == 'IND':
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'GRP':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'OTH':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
    elif target == 'UNT':
      f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')'''

In [5]:
df = pd.read_csv('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/bert_nli.csv', delimiter='\t')

In [8]:
df = df.sample(frac=1)

In [9]:
l = len(df)
train_df = df[:int(0.8*l)]
val_df = df[int(0.8*l):int(0.9*l)]
test_df = df[int(0.9*l):]

In [10]:
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

In [11]:
train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

In [12]:
val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

In [13]:
test_df['sentence1'] = test_df['sentence1'].astype(str)
test_df['sentence2'] = test_df['sentence2'].astype(str)

In [14]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
test_df = test_df[(test_df['sentence1'].str.split().str.len() > 0) & (test_df['sentence2'].str.split().str.len() > 0)]

In [15]:
train_df

,tweet_id,gold_label,sentence1,sentence2
16214,17281,contradiction,@USER you still doing music?,This is offensive speech.
12386,65031,contradiction,@USER She is so right!,This is offensive speech.
4808,40221,entailment,@USER Maybe there wouldn't be 5 of these thing...,This is offensive speech.
9790,66545,contradiction,@USER @USER You are insinuating that the rape ...,This is offensive speech.
8079,43348,contradiction,@USER I’ve always seen myself as a classical L...,This is offensive speech.
...,...,...,...,...
1700,23902,entailment,@USER This is not the first time she has expre...,This is targeted offense.
5118,80103,entailment,@USER He tihnks he is telling the big lie.,This is offensive speech.
4328,50253,contradiction,@USER my d better now! Prescott booty meat smh,This is offensive speech.
8726,26572,entailment,@USER URL LOOK ST THIS BULLSHIT,This is targeted offense.


In [16]:
val_df

,tweet_id,gold_label,sentence1,sentence2
3910,60343,contradiction,@USER @USER @USER @USER @USER Thank God they b...,This is offensive speech.
10385,20359,contradiction,@USER He has the msm covering up,This is offensive speech.
229,51815,contradiction,@USER @USER @USER The 2015 manifesto promised ...,This is offensive speech.
10944,61602,entailment,@USER @USER @USER Holy shit get a lawyer false...,This is offensive speech.
16063,58976,contradiction,@USER His gun control skills need work he miss...,This is offensive speech.
...,...,...,...,...
2520,55697,contradiction,@USER better leave #HIAC with #SDLIVE Women's ...,This is offensive speech.
5263,67690,contradiction,@USER @USER @USER @USER @USER *sigh* You are ...,This is offensive speech.
13423,24575,entailment,@USER lets stretch bitches,This is offensive speech.
140,88013,contradiction,@USER The president’s feelings are totally opp...,This is offensive speech.


In [17]:
test_df

,tweet_id,gold_label,sentence1,sentence2
4335,43257,contradiction,@USER Haley thanx! you know how to brighten up...,This is offensive speech.
11774,32533,contradiction,2) The IRS attacked conservatives. The FBI ros...,This is offensive speech.
16574,34147,contradiction,@USER @USER He is not good in Debate but very ...,This is offensive speech.
674,97751,contradiction,@USER ATTENTION! @USER is publicizing an ANTIF...,This is offensive speech.
1103,57111,contradiction,@USER When is LVMPD and MGM going to be held a...,This is offensive speech.
...,...,...,...,...
122,21379,entailment,@USER @USER @USER Please call trump out for wh...,This is targeted offense.
3447,29758,contradiction,@USER There she is nibbling in her lower lip a...,This is offensive speech.
4593,26995,contradiction,@USER @USER Give the kid a break! He is trying...,This is offensive speech.
8424,13473,entailment,@USER Watched the press conf with Trump and he...,This is targeted offense.


In [18]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df):
    self.label_dict = {'entailment': 0, 'contradiction': 1}

    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.base_path = '/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/mnli-data'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.init_data()

  def init_data(self):
    #Saving takes too much RAM
    
    if os.path.exists(os.path.join(self.base_path, 'train_data.pkl')):
      print("Found training data")
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'rb') as f:
        self.train_data = pickle.load(f)
    else:
      self.train_data = self.load_data(self.train_df)
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'wb') as f:
        pickle.dump(self.train_data, f)
    if os.path.exists(os.path.join(self.base_path, 'val_data.pkl')):
      print("Found val data")
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'rb') as f:
        self.val_data = pickle.load(f)
    else:
      self.val_data = self.load_data(self.val_df)
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'wb') as f:
        pickle.dump(self.val_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data.pkl')):
      print("Found test data")
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'rb') as f:
        self.test_data = pickle.load(f)
    else:
      self.test_data = self.load_data(self.test_df)
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'wb') as f:
        pickle.dump(self.test_data, f)
    # self.train_data = self.load_data(self.train_df)
    # self.val_data = self.load_data(self.val_df)
    # self.test_data = self.load_data(self.test_df)
  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader = DataLoader(
      self.test_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader, test_loader

In [19]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Found training data
Found val data
Found test data


In [20]:
train_loader, val_loader, test_loader = mnli_dataset.get_data_loaders(batch_size=16)

In [21]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [23]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,483,778 trainable parameters


In [25]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)
  for epoch in range(3, EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    torch.save(model.state_dict(),os.path.join('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Weights', 'nli_{}.pt'.format(epoch)))
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [28]:
import time
def test(model, test_loader):
  model.load_state_dict(torch.load('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Weights/nli_2.pt'))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  with torch.no_grad():
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      total_test_loss += loss.item()
      total_test_acc  += acc.item()

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
train(model, train_loader, val_loader, optimizer)

Epoch 1: train_loss: 0.3921 train_acc: 0.8164 | val_loss: 0.3685 val_acc: 0.8388
00:13:23.98
Epoch 2: train_loss: 0.2724 train_acc: 0.8842 | val_loss: 0.3548 val_acc: 0.8440
00:13:29.16
Epoch 3: train_loss: 0.1687 train_acc: 0.9394 | val_loss: 0.4651 val_acc: 0.8306
00:13:24.09


KeyboardInterrupt: ignored

In [29]:
test(model, test_loader)

test_loss: 0.5086 test_acc: 0.8154
00:00:32.89
